In [1]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from preprocess import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
def extract_hog_features(image):
    # Convert image to grayscale
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Compute HOG features
    features = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3,3)) #orig was cells_per_block=(2,2)
    return features

In [5]:
dataset_dir = "outresize500"

# List to store image paths and corresponding labels
image_paths = []
labels = []
hog_features=[]
from tqdm import tqdm
# Iterate through each font type directory
for i,font_type in enumerate(os.listdir(dataset_dir)):
    print("Processing font type ",font_type)
    counter=0
    font_type_dir = os.path.join(dataset_dir, font_type)
    font_label=i
    if os.path.isdir(font_type_dir):
        # Iterate through each image in the font type directory
        for image_file in tqdm (os.listdir(font_type_dir)):
            # counter+=1
            # if counter>1000:
            #     break;
            image_path = os.path.join(font_type_dir, image_file)
            # Read the image
            image = cv2.imread(image_path)
            # image = preprocess(image)
            # # print("Preprocessed image shape is",image.shape)
            # image= removeSkew(image)
            # print("Skew-Corrected image shape is ",image.shape)
            if image is not None:
                # Extract HOG features
                currHoG=extract_hog_features(image)
                hog_features.append(currHoG)
                # Add HOG features to the list
                image_paths.append(image_path)
                labels.append(font_label)
#convert hog_features to float 32
hog_features=np.array(hog_features).astype(np.float32)

Processing font type  IBM Plex Sans Arabic


  0%|          | 0/1000 [00:00<?, ?it/s]


Processing font type  Lemonada


  0%|          | 0/1000 [00:00<?, ?it/s]


Processing font type  Marhey


  0%|          | 0/1000 [00:00<?, ?it/s]


Processing font type  Scheherazade New


  0%|          | 0/1000 [00:00<?, ?it/s]


In [4]:
# get PCA 
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99)
pca.fit(hog_features)
hog_features_pca = pca.transform(hog_features)

In [ ]:
hog_features_pca.shape

(4000, 3231)

In [ ]:
# save in pkl 
import pickle
with open('hog_features_resize500_pca99_3blocks.pkl', 'wb') as f:
    pickle.dump(hog_features_pca, f)

NameError: name 'hog_features_pca' is not defined

In [30]:
with open('labels500_pca95.pkl', 'wb') as f:
    pickle.dump(labels, f)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(hog_features_pca, labels, test_size=0.2, random_state=42)


In [ ]:
# Convert lists to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)

In [ ]:
# Train SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [ ]:

# Predict on the test set
y_pred = svm_model.predict(X_test)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.59


In [18]:
import joblib

# Save the trained SVM model to a file
joblib.dump(svm_model, 'svm_model_linear_resize500_pca90.pkl')


#loaded_model = joblib.load('svm_model.pkl')

['svm_model_linear_resize500_pca90.pkl']

In [19]:
#rbf
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

SVC()

In [20]:
y_pred = svm_model.predict(X_test)

In [21]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.66375


In [26]:
# import grid search
from sklearn.model_selection import GridSearchCV
params_grid = {'C': [2, 5, 7, 10, 12],
          'gamma': [0.001, 0.002, 0.0005, 0.004],
          'kernel':['sigmoid'] }

#Create the GridSearchCV object
grid_clf = GridSearchCV(SVC(class_weight='balanced'), params_grid,verbose=3,refit=True)

#Fit the data with the best possible parameters
grid_clf = grid_clf.fit(X_train, y_train)

#Print the best estimator with it's parameters
print(grid_clf.best_estimator_)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END ..C=2, gamma=0.001, kernel=sigmoid;, score=0.620 total time=   7.7s
[CV 2/5] END ..C=2, gamma=0.001, kernel=sigmoid;, score=0.603 total time=   7.9s
[CV 3/5] END ..C=2, gamma=0.001, kernel=sigmoid;, score=0.641 total time=   7.6s
[CV 4/5] END ..C=2, gamma=0.001, kernel=sigmoid;, score=0.603 total time=   7.7s
[CV 5/5] END ..C=2, gamma=0.001, kernel=sigmoid;, score=0.628 total time=   7.9s
[CV 1/5] END ..C=2, gamma=0.002, kernel=sigmoid;, score=0.452 total time=   6.5s
[CV 2/5] END ..C=2, gamma=0.002, kernel=sigmoid;, score=0.411 total time=   6.6s
[CV 3/5] END ..C=2, gamma=0.002, kernel=sigmoid;, score=0.461 total time=   6.7s
[CV 4/5] END ..C=2, gamma=0.002, kernel=sigmoid;, score=0.409 total time=   7.6s
[CV 5/5] END ..C=2, gamma=0.002, kernel=sigmoid;, score=0.416 total time=   6.5s
[CV 1/5] END .C=2, gamma=0.0005, kernel=sigmoid;, score=0.639 total time=   8.5s
[CV 2/5] END .C=2, gamma=0.0005, kernel=sigmoid

KeyboardInterrupt: 